In [1]:
import numpy as np
import random
from scipy.spatial import distance

#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:
X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]

In [11]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [12]:
for Xtnn, ytnn in zip(X_train, y_train):
    X_train_pairs = []
    y_train_pair_lables = []
    dists = []
    dists_indexes = []
    for i,(Xtnn2, ytnn2) in enumerate (zip(X_train, y_train)):
        if (ytnn != ytnn2):
            dist = distance.euclidean(Xtnn, Xtnn2)
            dists.append(dist)
            dists_indexes.append(i)
    
    dists = np.array(dists)
    dists_indexes = np.array(dists_indexes)
    _indexes = np.argsort(dists)

    # Sorted distances and labels
    dists, dists_indexes = dists[_indexes], dists_indexes[_indexes] 
    for Xtnn2, ytnn2 in zip(X_train, y_train):
        if (ytnn == ytnn2):
            Xconcat = np.concatenate((Xtnn, Xtnn2), axis = None)
            X_train_pairs.append(Xconcat)
            y_train_pair_lables.append(0)
            
            for i in range (30):
                Xtnn3 = X_train[dists_indexes[i]]
                ytnn3 = y_train[dists_indexes[i]]                 

                Xconcat = np.concatenate((Xtnn, Xtnn3), axis = None)
                X_train_pairs.append(Xconcat)
                if(ytnn == ytnn3):
                    y_train_pair_lables.append(0)
                else:
                    y_train_pair_lables.append(1)

    y_train_pair_lables = np.array(y_train_pair_lables)
    X_train_pairs = np.array(X_train_pairs)
    if (X_train_pairs.shape[0] != 0):
            mlp.partial_fit(X_train_pairs, y_train_pair_lables)            

Iteration 1, loss = 0.27878595
Iteration 2, loss = 34.45741336
Iteration 3, loss = 10.41911067
Iteration 4, loss = 0.28919760
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 5, loss = 5.14995704
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 6, loss = 13.06643499
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7, loss = 14.38746259
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 8, loss = 9.89558913
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 9, loss = 4.11282254
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 10, loss = 0.72418612
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 11, loss = 1.03558758
Training loss di

Iteration 72, loss = 0.53327503
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 73, loss = 0.53448353
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 74, loss = 0.52779824
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 75, loss = 0.51773242
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 76, loss = 0.51136383
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 77, loss = 0.51792676
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 78, loss = 0.50991995
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 79, loss = 0.50665108
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 80, lo

Iteration 141, loss = 0.34053173
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 142, loss = 0.33917380
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 143, loss = 0.33782541
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 144, loss = 0.33620374
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 145, loss = 0.33670849
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 146, loss = 0.33475595
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 147, loss = 0.33334541
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 148, loss = 0.33288792
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 230, loss = 0.25433096
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 231, loss = 0.25379877
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 232, loss = 0.25320980
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 233, loss = 0.25255750
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 234, loss = 0.25359436
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 235, loss = 0.25168510
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 236, loss = 0.25054232
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 237, loss = 0.25054083
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 319, loss = 0.23624532
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 320, loss = 0.23525110
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 321, loss = 0.23446521
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 322, loss = 0.23347937
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 323, loss = 0.23255108
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 324, loss = 0.23174991
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 325, loss = 0.23079627
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 326, loss = 0.22992319
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

...

Iteration 7066, loss = 0.86491897
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7067, loss = 0.82151597
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7068, loss = 0.84511369
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7069, loss = 0.83449423
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7070, loss = 0.82156755
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7071, loss = 0.83865557
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7072, loss = 0.80958132
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7073, loss = 0.83026086
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7132, loss = 0.95276292
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7133, loss = 0.84026594
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7134, loss = 1.45968318
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7135, loss = 1.03676968
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7136, loss = 1.43247401
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7137, loss = 1.06381694
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7138, loss = 1.26876526
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7139, loss = 0.80964845
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7198, loss = 0.54708471
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7199, loss = 0.52803384
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7200, loss = 0.54188314
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7201, loss = 0.53029342
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7202, loss = 0.53056852
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7203, loss = 0.53324236
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7204, loss = 0.53891004
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7205, loss = 0.54049531
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7266, loss = 0.64027024
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7267, loss = 0.66188809
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7268, loss = 0.61597697
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7269, loss = 0.68526308
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7270, loss = 0.63945916
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7271, loss = 0.76022645
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7272, loss = 0.62339823
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7273, loss = 0.64501801
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7333, loss = 0.90614312
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7334, loss = 0.85058665
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7335, loss = 0.88218645
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7336, loss = 1.07860872
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7337, loss = 1.08759666
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7338, loss = 1.06569515
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7339, loss = 1.10282455
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7340, loss = 0.65670017
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [13]:
#y_train_pair_lables

In [14]:
#X_train_pairs

In [15]:
#X_train_pairs.shape

In [16]:
#from sklearn.neural_network import MLPRegressor
#mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
#                    solver='adam', verbose=10, tol=1e-4, random_state=1,
#                    learning_rate_init=.1)

In [17]:
#mlp.fit(X_train_pairs, y_train_pair_lables)

In [18]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [19]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, metric = mlp, parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                if metric == mlp:
                    dist = mlp.predict(np.concatenate((query, gallery)).reshape((1,4096)))[0]
                elif metric == 'minkowski':
                    dist = distance.minkowski(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')           
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        y_valid = np.array(y_valid)
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A) 
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

In [20]:
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = mlp,
                                       parameters = None)

Accuracies by Rank:
Rank  1  = 0.14% 	 Rank  2  = 0.14% 	 Rank  3  = 0.14% 	 Rank  4  = 0.21% 	 Rank  5  = 0.29%
Rank  6  = 0.29% 	 Rank  7  = 0.36% 	 Rank  8  = 0.36% 	 Rank  9  = 0.36% 	 Rank  10  = 0.43%
Rank  11  = 0.50% 	 Rank  12  = 0.50% 	 Rank  13  = 0.57% 	 Rank  14  = 0.57% 	 Rank  15  = 0.57%
Rank  16  = 0.57% 	 Rank  17  = 0.64% 	 Rank  18  = 0.64% 	 Rank  19  = 0.64% 	 Rank  20  = 0.64%
Rank  21  = 0.71% 	 Rank  22  = 0.79% 	 Rank  23  = 0.93% 	 Rank  24  = 1.00% 	 Rank  25  = 1.00%
Rank  26  = 1.14% 	 Rank  27  = 1.21% 	 Rank  28  = 1.21% 	 Rank  29  = 1.29% 	 Rank  30  = 1.50%
mAP = 0.34%
